In [1]:
!pip install transformers
!pip install torch
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.2-cp310-cp310-linux_x86_64.whl size=582715 sha256=8dfbec33d6c77fe0bf18c3c14285f0ff054a13af67a7f53282eae35c02877ae3
  Stored in directory: /root/.cache/pip/wheels/7a/d9/59/473fa56df8e39430eeda369500b4e7127f5b243ba24c3c4297
Successfully built annoy


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from annoy import AnnoyIndex

# Load pre-trained FinBERT model and tokenizer
model_name = 'ProsusAI/finbert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Load financial documents and queries
documents = pd.read_csv('financial_documents.csv')
queries = pd.read_csv('queries.csv')

# Define the number of dimensions for the embeddings
embedding_size = 768

# Define the number of trees for the Annoy index
num_trees = 50

# Define the size of the concatenated embeddings
concatenated_size = 2 * embedding_size

# Define the query tower
class QueryTower(torch.nn.Module):
    def __init__(self):
        super(QueryTower, self).__init__()
        self.model = model
    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            output = self.model(input_ids=input_ids, attention_mask=attention_mask)
            query_embedding = output[0][:, 0, :]
            return query_embedding

# Define the document tower
class DocumentTower(torch.nn.Module):
    def __init__(self):
        super(DocumentTower, self).__init__()
        self.model = model
    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            output = self.model(input_ids=input_ids, attention_mask=attention_mask)
            document_embedding = output[0][:, 0, :]
            return document_embedding

# Create the two-tower model
class TwoTower(torch.nn.Module):
    def __init__(self):
        super(TwoTower, self).__init__()
        self.query_tower = QueryTower()
        self.document_tower = DocumentTower()
        self.fc1 = torch.nn.Linear(concatenated_size, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 1)

    def forward(self, query_input_ids, query_attention_mask, document_input_ids, document_attention_mask):
        query_embedding = self.query_tower(query_input_ids, query_attention_mask)
        document_embedding = self.document_tower(document_input_ids, document_attention_mask)
        concatenated_embedding = torch.cat((query_embedding, document_embedding), dim=1)
        x = torch.relu(self.fc1(concatenated_embedding))
        x = torch.relu(self.fc2(x))
        similarity_score = torch.sigmoid(self.fc3(x))
        return similarity_score

# Define the training function
def train(model, optimizer, data_loader, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        query_input_ids = batch['query_input_ids'].to(device)
        query_attention_mask = batch['query_attention_mask'].to(device)
        document_input_ids = batch['document_input_ids'].to(device)
        document_attention_mask = batch['document_attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(query_input_ids, query_attention_mask, document_input_ids, document_attention_mask)
        loss = torch.nn.BCELoss()(outputs, labels)
        loss.backward()
        optimizer.step()